# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [67]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../data/Part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [68]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
df['Sex'] = df['Sex'].fillna('None')
mean_df = df.groupby(['Sex'])['Age'].mean().reset_index()
mode_df = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Sex'])['Age'].median().reset_index()
max_df = df.groupby(['Sex'])['Age'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Sex'])
temp = pd.merge(temp, median_df, how='left', on=['Sex'])
temp = pd.merge(temp, max_df, how='left', on=['Sex'])
temp.columns = ['Sex', 'Age_Sale_Mean', 'Age_Sale_Mode', 'Age_Sale_Median', 'Age_Sale_Max']
temp

,Sex,Age_Sale_Mean,Age_Sale_Mode,Age_Sale_Median,Age_Sale_Max
0,female,27.915709,24.0,27.0,63.0
1,male,30.726645,19.0,29.0,80.0


In [80]:
df = pd.merge(df, temp, how='left', on=['Sex'])
# df = df.drop(['Sex'] , axis=1)
df.head()

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [79]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
# num_features.append('Sex')
le = LabelEncoder()
df = df[num_features]
# df['Sex'] = le.fit_transform(df['Sex'])
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

10 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Age_Sale_Mean', 'Age_Sale_Mode', 'Age_Sale_Median', 'Age_Sale_Max', 'Sex']



,Pclass,Age,SibSp,Parch,Fare,Age_Sale_Mean,Age_Sale_Mode,Age_Sale_Median,Age_Sale_Max,Sex
0,3,22.0,1,0,7.2500,30.726645,19.0,29.0,80.0,1
1,1,38.0,1,0,71.2833,27.915709,24.0,27.0,63.0,0
2,3,26.0,0,0,7.9250,27.915709,24.0,27.0,63.0,0
3,1,35.0,1,0,53.1000,27.915709,24.0,27.0,63.0,0
4,3,35.0,0,0,8.0500,30.726645,19.0,29.0,80.0,1


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [73]:
df_minus = df.drop(['Age_Sale_Mean','Age_Sale_Median','Age_Sale_Max'], axis = 1)
df_minus.head()

,Pclass,Age,SibSp,Parch,Fare,Age_Sale_Mode,Sex
0,3,22.0,1,0,7.2500,19.0,1
1,1,38.0,1,0,71.2833,24.0,0
2,3,26.0,0,0,7.9250,24.0,0
3,1,35.0,1,0,53.1000,24.0,0
4,3,35.0,0,0,8.0500,19.0,1


In [76]:
# 原始特徵 + 邏輯斯迴歸

train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfg

0.7834073637476873

In [77]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfg

0.7834073637476873